In [1]:
import sys
print(sys.version_info)
assert sys.version_info >= (3, 7)

from packaging import version
import sklearn
print ("Scikit-Learn version: ", sklearn.__version__)
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

sys.version_info(major=3, minor=11, micro=4, releaselevel='final', serial=0)
Scikit-Learn version:  1.3.0


here we're trying to get the wrong labeled data (like "=" is labeled to "+")
BUT it is impossible to check all 17000 img and check them(we can, but... really?)
or we can use the model to identify them

how? by using well-trained model! we have Mnist!
so the plan is;
1. train the mnist that has a good score(like 0.97 or 98)
2. for each img check the score using our mnist model. If the score is quite low(to a certain threshold, we may assume that is it mislabeled
3. check the img and label, and fix

this may minimize our man-work

In [2]:
from pathlib import Path

path = Path() / "npz" / "asdf"
print(path)

npz\asdf


## dataset preparation

In [4]:
from sklearn.datasets import fetch_openml

openml = fetch_openml('mnist_784', as_frame=False, parser="auto")
mnist = openml

# get the latest datasets - rebased
curr_data_num_trVal = np.load(path / "digit_data_TrVal_rebased.npz")
curr_data_sym_trVal = np.load(path / "op_data_TrVal_rebased.npz")

curr_data_num_test = np.load(path / "digit_data_10_Test_rebased.npz")
curr_data_sym_test = np.load(path / "op_data_10_Test_rebased.npz")

## prepare model

currently, our strongest model so far is the assemble model, with 0.97 score